In [3]:
import random
import re
import glob

In [1]:
instruction_set = ['AND' , 'ADD','ADC','BIC','CMN','CMP','EOR','ORR'
                   ,'RSB','RSC','SUB','SBC','MUL','MLA','MOV','MVN' , 'TST']

In [2]:
existing_inst = ['ADC' , 'ADD','ADR','AND','BIC','BL','BL','BX','B','CDP','CMN','EOR','FLDM','LDC','LDM','LDR','MCR','MLA'
                ,'MOVT','MOV','MRC','MUL','MVN','NOP','ORR','RET','RSB','RSC','SBC','SMULL','SMULTT','STC','STM','STR'
                 ,'SUB','SVC','SWP','TEQ','TST','UMULL','UMLAL','VMOV']

In [5]:
#RULES
def check_obfuscation(item):
    rules_trued = []
    surgary = item.split()
    #rule 1
    if surgary[0].startswith('MOV'):
        if 'R' in surgary[1] and surgary[2]=='#0':
            rules_trued.append(1)
    #rule 2
    if surgary[0].startswith('MOV') and 'R' in surgary[1] and '#' in surgary[2]:
        rules_trued.append(2)
    
    #rule 3
    if surgary[0].startswith('MOV') and 'R' in surgary[1] and '#' in surgary[2]:
        rules_trued.append(3)
    
    #rule 4
    if surgary[0].startswith('MUL') and 'R' in surgary[1] and  'R' in surgary[2] and  'R' in surgary[3]:
        rules_trued.append(4)
    
    #rule 5
    if len(surgary)==3 and surgary[0].startswith('EOR') and surgary[2]==surgary[3]:
        rules_trued.append(5)
    
    #rule 6
    if surgary[0].startswith('MVN') and surgary[1].startswith('R') and surgary[2].startswith('R'):
        rules_trued.append(6)
    
    #rule 7
    if surgary[0].startswith('CMP'):
        rules_trued.append(7)
    
    #rule 8
    if surgary[0].startswith('MVN') and surgary[1].startswith('R') and surgary[2].startswith('R'):
        rules_trued.append(8)
    
    #rule 9
    if len(surgary)==3 and surgary[0].startswith('TST') and '#' in surgary[2]:
        rules_trued.append(9)
    return rules_trued
    

In [6]:
#obfuscation
def do_obfuscate(rule_num):
    opcode_obfuscated = []
    if rule_num==1:
        opcode_obfuscated.append('EOR')
        return opcode_obfuscated
    elif rule_num==2:
        opcode_obfuscated.append('EOR')
        opcode_obfuscated.append('ADD')
        return opcode_obfuscated
    elif rule_num==3:
        opcode_obfuscated.append('MOV')
        opcode_obfuscated.append('ADD')
        return opcode_obfuscated
    elif rule_num==4:
        opcode_obfuscated.append('MLA')
        return opcode_obfuscated
    elif rule_num==5:
        opcode_obfuscated.append('MOV')
        return opcode_obfuscated
    elif rule_num==6:
        opcode_obfuscated.append('MOV')
        opcode_obfuscated.append('BIC')
        return opcode_obfuscated
    elif rule_num==7:
        opcode_obfuscated.append('SUB')
        return opcode_obfuscated
    elif rule_num==8:
        opcode_obfuscated.append('MOV')
        opcode_obfuscated.append('ORN')
        return opcode_obfuscated
    elif rule_num==9:
        opcode_obfuscated.append('AND')
        return opcode_obfuscated

In [7]:
lamba=.8 # 80% of codes  which satisfied the rules
obtained_data= []
dir='seprated_asm' # Address of current directory
for file in glob.glob(dir+'\\*.asm'): 
    file_asm  = open(file)
    data = file_asm.readlines()
    
    for i in range(len(data)):
        data[i] = re.sub(r'\s', ' ', data[i])
    
    counter=-1
    codes_for_obfuscation = []
    for item in data:
        counter+=1
        item = item.replace(",", "")
        sub_code = item.split()
        if not sub_code:
            continue
        for ins_ in instruction_set:
            if  ins_  not in sub_code[0]:
                continue
            rule_trued = check_obfuscation(item)
            if len(rule_trued) != 0:
                codes_for_obfuscation.append(counter)   
        
    data_copy = data[:]
    number_of_inculded = int(len(codes_for_obfuscation) * lamba)
    selected_indexes_data = []
    for i in range(number_of_inculded):
        selected_index = random.choice(codes_for_obfuscation)
        codes_for_obfuscation.remove(selected_index)
        selected_indexes_data.append(selected_index)

    selected_indexes_data.sort(reverse=True)
    for item in selected_indexes_data:
        possible_obfuscations = check_obfuscation(data[item])
        reule_selected = random.choice(possible_obfuscations)
        opcode_relaced_list = []
        opcode_relaced_list = do_obfuscate(reule_selected)
        data_copy[item] = opcode_relaced_list[0]
        del opcode_relaced_list[0]
        for i in range(0,len(opcode_relaced_list)):
            data_copy.insert(item+i+1 , opcode_relaced_list[i])

    extracted_data=[]
    for item in data_copy:
        item = item.replace(",", "")
        sub_code = item.split()
        if not sub_code:
            continue
        for ins_ in existing_inst:
            if sub_code[0].startswith(ins_):
                extracted_data.append(ins_)
                continue
    sample_obf = ''
    for item in extracted_data:
        sample_obf = sample_obf + ' ' +item
#     obtained_data.append(sample_obf)
    
    file_name = file.split('\\')[-1]
    thefile = open('obfuscated-80\\'+file_name.split('.')[0]+'.txt', 'w')
    thefile.write("%s\n" % sample_obf)
    thefile.close()

In [8]:
thefile = open('100obfuscated_malwares.txt', 'w')
for item in obtained_data:
    thefile.write("%s\n" % item)
thefile.close()

In [8]:
# lambas = [.2,.4,.6,.8,1]
lamba=1
data_copy = data[:]
number_of_inculded = int(len(codes_for_obfuscation) * lamba)
selected_indexes_data = []
for i in range(number_of_inculded):
    selected_index = random.choice(codes_for_obfuscation)
    codes_for_obfuscation.remove(selected_index)
    selected_indexes_data.append(selected_index)

selected_indexes_data.sort(reverse=True)
for item in selected_indexes_data:
    possible_obfuscations = check_obfuscation(data[item])
    reule_selected = random.choice(possible_obfuscations)
    opcode_relaced_list = []
    opcode_relaced_list = do_obfuscate(reule_selected)
    data_copy[item] = opcode_relaced_list[0]
    del opcode_relaced_list[0]
    for i in range(0,len(opcode_relaced_list)):
        data_copy.insert(item+i+1 , opcode_relaced_list[i])

In [4]:
counter=-1
codes_for_obfuscation = []
for item in data:
    counter+=1
    item = item.replace(",", "")
    sub_code = item.split()
    if not sub_code:
        continue
    for ins_ in instruction_set:
        if  ins_  not in sub_code[0]:
            continue
        rule_trued = check_obfuscation(item)
        if len(rule_trued) != 0:
            codes_for_obfuscation.append(counter)

NameError: name 'data' is not defined

In [12]:
extracted_data=[]
for item in data_copy:
    item = item.replace(",", "")
    sub_code = item.split()
    if not sub_code:
        continue
    for ins_ in existing_inst:
        if sub_code[0].startswith(ins_):
            extracted_data.append(ins_)
            continue